# Sentimental analysis of Covid19 of time 

In [105]:
import tweepy as ty
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

from textblob import Word, TextBlob



In [106]:
#Twitter API credentials, keys and tokens from the Twitter Dev Console 
consumer_key = 'kN1fqEXeaDrXIOPRHbJOJbq7S'
consumer_secret = 'l52epNKTPsj42NBq5xL7x02PP8tmw4HyJy4zeDIC42MtH9cVf4'
access_token = '1347602020665065474-UpprxnN6XCceaLPfyEVcR9JQDMgT0t'
access_token_secret = 'zPRQGjY0LdeKcn1hrxrj6sb5HcWXGy2j4RNFGIbeM8dc2'


In [107]:
# create OAuthHandler object 
auth = ty.OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(access_token, access_token_secret) 

# create API object to fetch tweets with auth info

api = ty.API(auth,wait_on_rate_limit=True) 

In [108]:
hashtag = "#covid19"
query = ty.Cursor(api.search, q=hashtag,lang = "en").items(1000)
tweets = [{'Tweets':tweet.text,'Timestamp':tweet.created_at}for tweet in query]




In [109]:
df = pd.DataFrame.from_dict(tweets)
df.head()

,Tweets,Timestamp
0,This is infuriating. The “reopen NM” shenaniga...,2021-01-24 23:41:00
1,The #COVID19 pandemic has been horrific. But ...,2021-01-24 23:40:59
2,RT @press4change: 🤦‍♂️Denmark’s CDC is #genome...,2021-01-24 23:40:59
3,RT @DrEricDing: Mulling 🇬🇧 disclosure that new...,2021-01-24 23:40:58
4,RT @JamesHesp: If the #COVID19 vaccine is mean...,2021-01-24 23:40:57


In [110]:
import nltk
from nltk.corpus import stopwords

from textblob import Word, TextBlob


nltk.download("stopwords")
nltk.download("wordnet")
stop_words = stopwords.words("english")
custom_stopwords =["RT","#covid19"]



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/spandanagali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/spandanagali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [111]:
#cleaning of the tweets : 
def clean_tweets(tweet, custom_stopwords):
    tweet = re.sub(r'@[A-Za-z0-9]+','',tweet)
    tweet = re.sub(r':','',tweet)
    tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    clean_tweet = tweet
    clean_tweet.replace('[^\w\s]', '')
    
    clean_tweet = " ".join(word for word in clean_tweet.split() if word not in stop_words)
    clean_tweet = " ".join(word for word in clean_tweet.split() if word not in custom_stopwords)
    clean_tweet = " ".join(Word(word).lemmatize() for word in clean_tweet.split())
    return(clean_tweet)

df['clean Tweet'] = df['Tweets'].apply(lambda x: clean_tweets(x, custom_stopwords))
df.head()

,Tweets,Timestamp,clean Tweet
0,This is infuriating. The “reopen NM” shenaniga...,2021-01-24 23:41:00,This infuriating. The “reopen NM” shenanigan c...
1,The #COVID19 pandemic has been horrific. But ...,2021-01-24 23:40:59,The #COVID19 pandemic horrific. But match Blac...
2,RT @press4change: 🤦‍♂️Denmark’s CDC is #genome...,2021-01-24 23:40:59,🤦‍♂️Denmark’s CDC #genomesequencing *every sin...
3,RT @DrEricDing: Mulling 🇬🇧 disclosure that new...,2021-01-24 23:40:58,Mulling 🇬🇧 disclosure new #B117 variant could ...
4,RT @JamesHesp: If the #COVID19 vaccine is mean...,2021-01-24 23:40:57,If #COVID19 vaccine meant saviour rotten situa...


In [112]:
# Sentimental analysis 
df['polarity'] = df['clean Tweet'].apply(lambda x: TextBlob(x).sentiment[0])
df['subjectivity'] = df['clean Tweet'].apply(lambda x: TextBlob(x).sentiment[1])
df[['clean Tweet','polarity', 'subjectivity']].head(20)




,clean Tweet,polarity,subjectivity
0,This infuriating. The “reopen NM” shenanigan c...,-0.450000,0.600000
1,The #COVID19 pandemic horrific. But match Blac...,-0.455556,0.477778
2,🤦‍♂️Denmark’s CDC #genomesequencing *every sin...,-0.071429,0.214286
3,Mulling 🇬🇧 disclosure new #B117 variant could ...,0.018182,0.427273
4,If #COVID19 vaccine meant saviour rotten situa...,0.000000,0.400000
5,I've watched many elderly people suffer prison...,0.366667,0.466667
6,OH SHIT—Outbreak infects 124 127 nursing home ...,-0.166667,0.166667
7,This time need country working together end #C...,0.000000,0.000000
8,"The tragedy continues! 💔 Yassin Dabeh, cleaner...",0.100000,0.400000
9,#Fauci deserves utmost respect 4 staying cours...,0.000000,0.000000


In [113]:
#Word cloud
allWords = ''.join([twts for twts in df['clean Tweet']])
wordCloud=WordCloud(width = 1000, height=1000, random_state = 2, max_font_size = 119).generate(allWords)

plt.imshow(wordCloud,interpolation="bilinear")
plt.axis("off")
plt.show() #pop-out plot




In [114]:
# negative, positive and neutaral 
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
    
df['Sentiment']= df['polarity'].apply(analysis)

df

,Tweets,Timestamp,clean Tweet,polarity,subjectivity,Sentiment
0,This is infuriating. The “reopen NM” shenaniga...,2021-01-24 23:41:00,This infuriating. The “reopen NM” shenanigan c...,-0.450000,0.600000,Negative
1,The #COVID19 pandemic has been horrific. But ...,2021-01-24 23:40:59,The #COVID19 pandemic horrific. But match Blac...,-0.455556,0.477778,Negative
2,RT @press4change: 🤦‍♂️Denmark’s CDC is #genome...,2021-01-24 23:40:59,🤦‍♂️Denmark’s CDC #genomesequencing *every sin...,-0.071429,0.214286,Negative
3,RT @DrEricDing: Mulling 🇬🇧 disclosure that new...,2021-01-24 23:40:58,Mulling 🇬🇧 disclosure new #B117 variant could ...,0.018182,0.427273,Positive
4,RT @JamesHesp: If the #COVID19 vaccine is mean...,2021-01-24 23:40:57,If #COVID19 vaccine meant saviour rotten situa...,0.000000,0.400000,Neutral
...,...,...,...,...,...,...
995,RT @SGT_921: Another Covid patrol finished and...,2021-01-24 23:29:09,_921 Another Covid patrol finished another hou...,0.050000,0.700000,Positive
996,RT @DavidLat: 5/ Here's the tweet from @ASUFed...,2021-01-24 23:29:07,5/ Here's tweet . They're hosting discussion J...,0.000000,0.000000,Neutral
997,RT @NaheedD: To all those who say #COVID19 onl...,2021-01-24 23:29:06,To say #COVID19 cause severe illness &amp; dea...,0.166667,0.333333,Positive
998,Tom Brady refuses to wear a mask. At least we ...,2021-01-24 23:29:06,Tom Brady refuse wear mask. At least see didn’...,0.100000,0.450000,Positive


In [116]:
#plot

for i in range(0,df.shape[0]):
    plt.scatter(df['polarity'][i],df['subjectivity'][i],color='Green')
    
plt.title('sentiment Analysis of covid 19')
plt.xlabel('Polarity')
plt.ylabel('subjectivity')
plt.show()

In [117]:
# percentage of positive or negative 

positive_twt = df[df.Sentiment == 'Positive']
negative_twt = df[df.Sentiment == 'Negative']
neutral_twt = df[df.Sentiment == 'Neutral']

ppercent=round( (positive_twt.shape[0]/df.shape[0])*100,1)
npercent=round( (negative_twt.shape[0]/df.shape[0])*100,1)
nnpercent=round( (neutral_twt.shape[0]/df.shape[0])*100,1)

print("Positive :" + str(ppercent) + "%")
print("Negative :" + str(npercent) + "%")
print("Neutral :" + str(nnpercent) + "%")





Positive :45.8%
Negative :13.8%
Neutral :40.4%


In [118]:
df['Sentiment'].value_counts()
df['Sentiment'].value_counts().plot(kind="bar")
plt.show()